In [9]:
import sqlalchemy
import pandas as pd
import yahooquery as yq
import sqlite3 as lite
from dateutil.relativedelta import relativedelta
import numpy as np
import datetime as dt
import talib as ta

In [3]:
engine = sqlalchemy.create_engine('sqlite:///' + "SEC_Filings.db", execution_options={"sqlite_raw_colnames": True})
df_ticker = pd.read_sql_table('Ticker_Table', engine)
df_fin_bal = pd.DataFrame()
df_fin_inc = pd.DataFrame()
df_fin_stock = pd.DataFrame()
bal_vars = ['asOfDate', 'CashAndCashEquivalents','TotalAssets','TotalDebt','TangibleBookValue','OrdinarySharesNumber']
inc_vars = ['asOfDate', 'BasicEPS']
stock_vars = ['date', 'adjclose']
# bal_sheet_error = []
bal_sheet_error_tickers = []
inc_stat_error_tickers = []

for ticker in df_ticker['Ticker']:
    tick = yq.Ticker(ticker)
    df_bal_tick = tick.balance_sheet(frequency='a', trailing=False)
    df_inc_tick = tick.income_statement(frequency='a', trailing=False)
    try: # some companies without pricing info included in list
        df_stock_tick = tick.history(interval='1d', start='2019-04-01').reset_index(level=1)
        df_stock_tick['date'] = df_stock_tick['date'] - relativedelta(days=1) 
    except:
        continue
    try:
        df_bal_tick = df_bal_tick[bal_vars]
        df_fin_bal = pd.concat([df_fin_bal, df_bal_tick], axis=0)

    except:
        if type(df_bal_tick) == pd.DataFrame:
            # bal_sheet_error.append(f'{ticker}: Missing columns {[var for var in bal_vars if var not in df_bal.columns]}')
            bal_sheet_error_tickers.append(ticker)
        else:
            # bal_sheet_error.append(f'{ticker} Balance sheet not Available')
            bal_sheet_error_tickers.append(ticker)

    try:
        df_inc_tick = df_inc_tick[inc_vars]
        df_fin_inc = pd.concat([df_fin_inc, df_inc_tick], axis=0)

    except:
        if type(df_inc_tick) == pd.DataFrame:
            # print([var for var in inc_vars if var not in df_inc.columns])
            inc_stat_error_tickers.append(ticker)
        else:
            inc_stat_error_tickers.append(ticker)

    df_stock_tick = df_stock_tick[stock_vars]
    df_fin_stock = pd.concat([df_fin_stock, df_stock_tick], axis=0)

            
df_fin_bal.reset_index(drop=False, inplace=True)
df_fin_inc.reset_index(drop=False, inplace=True)
df_fin_stock.reset_index(drop=False, inplace=True)

In [4]:
# Remove companies with 0 assets or 0 cash
df_fin_bal = df_fin_bal.loc[(df_fin_bal['TotalAssets']!=0) & (df_fin_bal['CashAndCashEquivalents'])!=0]
df_fin_inc.fillna(value=0, inplace=True)

# Only take rows with if matched in both dataframes
df_fin_joined = df_fin_inc.merge(df_fin_bal, how='inner', on=['symbol', 'asOfDate'])

In [5]:
df_fin_stock = pd.DataFrame()

for ticker in df_ticker['Ticker']:
    tick = yq.Ticker(ticker)
    try: # some companies without pricing info included in list
        df_stock_tick = tick.history(interval='1mo', start='2019-04-01').reset_index(level=1)
        df_stock_tick['date'] = df_stock_tick['date'] - relativedelta(days=1) 
    except:
        continue
    df_stock_tick = df_stock_tick[stock_vars]
    df_fin_stock = pd.concat([df_fin_stock, df_stock_tick], axis=0)
df_fin_stock.reset_index(drop=False, inplace=True)

df_fin_stock = df_fin_stock.loc[df_fin_stock['date'].apply(type)==dt.date]
df_fin_stock['date'] = df_fin_stock['date'].astype('datetime64[ns]')

df_fin_joined = df_fin_joined.merge(df_fin_stock, how='inner', left_on=['symbol', 'asOfDate'], right_on=['symbol', 'date'])

In [44]:
df_fin_cleaned = pd.DataFrame()
# backfill missing values based on company
for ticker in df_fin_joined['symbol'].unique():
    df_fin_cleaned = pd.concat([df_fin_cleaned, df_fin_joined.loc[df_fin_joined['symbol']==ticker].sort_values(
                        by='asOfDate', ascending=True).fillna(method='ffill')])

# Remove rows with incomplete data
missing_ticks = df_fin_cleaned[df_fin_cleaned.isnull().any(axis=1)]['symbol'].unique()
df_fin_cleaned = df_fin_cleaned.loc[~df_fin_cleaned['symbol'].isin(missing_ticks)].reset_index(drop=True)

df_fin_cleaned = df_ticker.merge(df_fin_cleaned, how="right", left_on="Ticker", right_on="symbol").drop("symbol", axis=1)

In [45]:
df_fin_cleaned['MarketCap'] = df_fin_cleaned['OrdinarySharesNumber'] * df_fin_cleaned['adjclose']
df_fin_cleaned['BTM'] = df_fin_cleaned['TangibleBookValue']/df_fin_cleaned['MarketCap']
df_fin_cleaned['Gearing'] = df_fin_cleaned['TotalDebt']/df_fin_cleaned['TotalAssets']
df_fin_cleaned['CashToAssets'] = df_fin_cleaned['CashAndCashEquivalents']/df_fin_cleaned['TotalAssets']
df_fin_cleaned['logMktCap'] = df_fin_cleaned['MarketCap'].apply(np.log)
df_fin_cleaned.drop(['CashAndCashEquivalents', 'TotalAssets', 'TotalDebt', 'TangibleBookValue', 'OrdinarySharesNumber', 'adjclose', 'MarketCap'], axis=1, inplace=True)

In [11]:
# Relative Strength Index (RSI) using Talib
df_val = pd.DataFrame()

df_tickers = df_ticker['Ticker']
df_tickers.reset_index(drop=True, inplace=True)

for ticker in df_tickers.unique():
    df_adjclose = yq.Ticker(ticker).history(start='2017-9-30', interval='1d')
    try:
        rsi = ta.RSI(df_adjclose['adjclose'])
    except:
        continue
    df_rsi = pd.DataFrame()
    df_rsi['RSI'] = rsi
    df_rsi.reset_index(inplace=True)
    df_rsi['date'] = pd.to_datetime(df_rsi['date'], utc=True)
    df_rsi['date'] = df_rsi['date'].dt.year
    df_rsi = df_rsi.drop_duplicates(subset=['date'], keep='last', ignore_index=True, inplace=False)
    df_val = pd.concat([df_val, df_rsi], axis=0)

df_val = df_val.loc[df_val['date'] > 2018]

df_val.reset_index(drop=True, inplace=True)
df_val.rename(columns={'symbol': 'Ticker'}, inplace=True)
df_val

,symbol,date,RSI
0,ESQ,2019,59.478684
1,ESQ,2020,49.872718
2,ESQ,2021,47.938937
3,ESQ,2022,52.611254
4,ESQ,2023,45.474529
...,...,...,...
2490,PME,2019,48.143150
2491,PME,2020,40.754503
2492,PME,2021,41.279895
2493,PME,2022,43.769206


In [46]:
df_val.rename(columns={'symbol': 'Ticker'}, inplace=True)

df_fin_cleaned['date'] = df_fin_cleaned['date'].dt.year
df_fin_final = df_val.merge(df_fin_cleaned, how='inner', left_on=['Ticker', 'date'], right_on=['Ticker', 'date'])
df_pop_rsi = df_fin_final.pop('RSI')
df_fin_final.insert(10, 'RSI', df_pop_rsi)
df_fin_final

,Ticker,date,CIK,Company,asOfDate,BasicEPS,BTM,Gearing,CashToAssets,logMktCap,RSI
0,ESQ,2019,0001531031,"Esquire Financial Holdings, Inc.",2019-12-31,1.91,0.644013,0.000108,0.077450,18.965636,59.478684
1,ESQ,2020,0001531031,"Esquire Financial Holdings, Inc.",2020-12-31,1.70,0.739600,0.000052,0.069589,18.954042,49.872718
2,ESQ,2021,0001531031,"Esquire Financial Holdings, Inc.",2021-12-31,2.40,0.520391,0.000042,0.126535,19.436657,47.938937
3,ESQ,2022,0001531031,"Esquire Financial Holdings, Inc.",2022-12-31,0.00,0.415587,0.000035,0.117596,19.757168,52.611254
4,PGC,2019,0001050743,PEAPACK GLADSTONE FINANCIAL CORP,2019-12-31,2.46,0.856992,0.064933,0.040148,20.107702,57.715730
...,...,...,...,...,...,...,...,...,...,...,...
1567,SHMP,2021,0001465470,NaturalShrimp Inc,2021-03-31,-0.01,0.009387,0.533187,0.010238,19.633952,56.147951
1568,SHMP,2022,0001465470,NaturalShrimp Inc,2022-03-31,-0.16,-0.341639,0.096214,0.045755,18.801838,28.835091
1569,PME,2019,0001517130,Pingtan Marine Enterprise Ltd.,2019-12-31,0.07,1.589562,0.563671,0.024973,18.262616,48.143150
1570,PME,2020,0001517130,Pingtan Marine Enterprise Ltd.,2020-12-31,-0.92,0.589988,0.728327,0.001493,18.587555,40.754503


In [48]:
# Add to database
conn = lite.connect('Financial.db')
conn.close()
engine_fin = sqlalchemy.create_engine('sqlite:///' + 'Financial.db', execution_options={"sqlite_raw_colnames": True})
df_fin_cleaned.to_sql("FinancialDataJoined", engine_fin, if_exists='replace', index=False)

1576

In [49]:
# Add to database
conn = lite.connect('Financial.db')
conn.close()
df_fin_final.to_sql("Financial_Final_Data", engine_fin, if_exists='replace', index=False)

1572